In [1]:
import pandas as pd
import numpy as np

In [ ]:
#Merge service data with unique customer group
cust_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/cust_grp_noused_1.csv', low_memory=False)
serv_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/serv_noused_1.csv', low_memory=False)
#Check data sanity
display(len(cust_grp_1.unique_cust_grp.unique()))
display(len(cust_grp_1.repm_vin.unique()))
display(len(serv_grp_1.repm_vin.unique()))
display(serv_grp_1.shape)
cust_grp_1_dict = pd.Series(cust_grp_1['unique_cust_grp'].values, index=cust_grp_1['repm_vin']).to_dict()
serv_grp_1['unique_cust_grp'] = serv_grp_1.repm_vin.map(cust_grp_1_dict)
display(serv_grp_1.shape)

In [ ]:
#Merge with car age
del(cust_grp_1)
grp_1_vin_age = pd.read_csv('D:/Referral/Cust_grp_splits/serv_noused_1_vin_age.csv', low_memory=False)
vin_age_map = pd.Series(grp_1_vin_age.vinm_model_year.values,index=grp_1_vin_age.repm_vin).to_dict()
serv_grp_1['car_model_year'] = serv_grp_1.repm_vin.map(vin_age_map)
display(serv_grp_1.shape)
display(serv_grp_1.columns)
display(serv_grp_1.car_model_year.isnull().sum())
serv_grp_1['car_age_years'] = (2019 - serv_grp_1['car_model_year']) #Calculate car age replace 0 years with 1 year
serv_grp_1['car_age_years'] = np.where(serv_grp_1['car_age_years']==0, 1, serv_grp_1['car_age_years'])
serv_grp_1 = serv_grp_1.drop_duplicates(subset=['repm_dlr_code', 'repm_cust_no', 'repm_vin', 'repm_ro_dtime',
                                  'repm_work_type', 'repm_milg'])
display(serv_grp_1.shape)
serv_grp_1 = serv_grp_1[serv_grp_1.car_model_year.notnull()] #Remove rows where car age is not present
display(serv_grp_1.shape

In [ ]:
serv_grp_1['repm_ro_dtime'] = pd.to_datetime(serv_grp_1['repm_ro_dtime'])
serv_grp_1 = serv_grp_1.sort_values(['unique_cust_grp', 'repm_vin', 'repm_work_type', 'repm_ro_dtime'], ascending=[True, 
                                                                                                                  True, True,
                                                                                                                  False])
display(serv_grp_1.repm_work_type.value_counts())
display(serv_grp_1.head())
serv_grp_1['day_diff'] = serv_grp_1.sort_values(['repm_vin', 'repm_ro_dtime'], ascending=[True,False]).groupby(
    ['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))
serv_grp_1['day_diff'] = serv_grp_1['day_diff'].dt.days

serv_grp_1_x = serv_grp_1.groupby('repm_vin')['repm_ro_dtime'].apply(lambda x: x.max()-x.min()).reset_index().rename(
    columns={'repm_ro_dtime': 'service_age_vin'})
serv_grp_1_x['service_age_vin'] = serv_grp_1_x['service_age_vin']/np.timedelta64(1,'D')
serv_grp_1 = pd.merge(serv_grp_1, serv_grp_1_x, how='left', on=['repm_vin'])
del(serv_grp_1_x)

def serv_hist(df):
    df['Diff_estimate_bill'] = df['repm_estmt_amt'] - df['billable_amount']
    df['Milg_diff_vin'] = df['repm_milg'] - df['repm_prvs_milg']
    df['total_service_count_vin'] = df.repm_vin.map(df.groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['FS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='FS')|
                                            (df['repm_work_type']=='UF')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['FS_count_vin_norm'] = df['FS_count_vin']/df['car_age_years']
    df['PS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='PS')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['PS_count_vin_norm'] = df['PS_count_vin']/df['car_age_years']
    df['RR_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='RR')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['RR_count_vin_norm'] = df['RR_count_vin']/df['car_age_years']
    df['AR_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='AR')|
                            (df['repm_work_type']=='RA')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['AR_count_vin_norm'] = df['AR_count_vin']/df['car_age_years']
    df['OC_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='OC')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['OC_count_vin_norm'] = df['OC_count_vin']/df['car_age_years']
    return df

serv_grp_1_tst = serv_hist(serv_grp_1)

In [ ]:
serv_grp_1_tst = serv_grp_1_tst.sort_values(['unique_cust_grp', 'repm_vin', 'repm_ro_dtime'], ascending=[True, 
                                                                                                 True, False])
                                                                                                                 
serv_grp_1_tst['FS_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='FS')|
                                               (serv_grp_1_tst['repm_work_type']=='UF')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['PS_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='PS')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['RR_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='RR')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['OC_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='OC')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['AR_mlg_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='AR')|
                                             (serv_grp_1_tst['repm_work_type']=='RA')].groupby(['repm_vin'])['repm_milg'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['FS_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='FS')|
                                               (serv_grp_1_tst['repm_work_type']=='UF')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['PS_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='PS')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['RR_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='RR')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['OC_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='OC')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['AR_day_diff'] = serv_grp_1_tst[(serv_grp_1_tst['repm_work_type']=='AR')|
                                             (serv_grp_1_tst['repm_work_type']=='RA')].groupby(['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))

serv_grp_1_tst['FS_day_diff'] = serv_grp_1_tst['FS_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['PS_day_diff'] = serv_grp_1_tst['PS_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['RR_day_diff'] = serv_grp_1_tst['RR_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['OC_day_diff'] = serv_grp_1_tst['OC_day_diff']/np.timedelta64(1,'D')
serv_grp_1_tst['AR_day_diff'] = serv_grp_1_tst['AR_day_diff']/np.timedelta64(1,'D')

def flat(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df

serv_feat_noused_1 = serv_grp_1_tst.groupby('unique_cust_grp').agg({'FS_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                             'PS_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                             'RR_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                              'OC_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                             'AR_mlg_diff': ['mean', 'median', 'min', 'max'],
                                                              'Milg_diff_vin':['mean', 'median', 'min', 'max'],
                                                              'billable_amount': ['mean', 'median', 'min', 'max'],
                                                             'Diff_estimate_bill': ['mean', 'median', 'min', 'max'],                                                                    
                                                             'FS_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'PS_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'RR_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'OC_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'AR_day_diff': ['mean', 'median', 'min', 'max'],
                                                             'FS_count_vin_norm':['mean', 'median', 'min', 'max'],
                                                             'PS_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'RR_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'OC_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'AR_count_vin_norm': ['mean', 'median', 'min', 'max'],
                                                             'FS_count_vin':['mean', 'median', 'min', 'max'],
                                                             'PS_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'RR_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'OC_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'AR_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'repm_vin': pd.Series.nunique,
                                                             'car_age_years': 'mean',
                                                             'service_age_vin': 'mean'
                                                             }).reset_index()
serv_feat_noused_1 = flat(serv_feat_noused_1)
display(serv_feat_noused_1.head())

serv_feat_noused_1.to_csv('D:/Referral/Cust_grp_splits/serv_feat_noused_1.dsv', encoding='utf-8', index=False)

**Repeat visit and recent visit analysis**

In [ ]:
cust_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/cust_grp_noused_1.csv', low_memory=False)
serv_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/serv_noused_1.csv', low_memory=False)
cust_grp_1_dict = pd.Series(cust_grp_1['unique_cust_grp'].values, index=cust_grp_1['repm_vin']).to_dict()
serv_grp_1['unique_cust_grp'] = serv_grp_1.repm_vin.map(cust_grp_1_dict)
del(cust_grp_1)
serv_grp_1 = serv_grp_1.drop_duplicates(subset=['repm_dlr_code', 'repm_cust_no', 'repm_vin', 'repm_ro_dtime',
                                  'repm_work_type', 'repm_milg'])
serv_grp_1['repm_ro_dtime'] = pd.to_datetime(serv_grp_1['repm_ro_dtime'])
serv_grp_1 = serv_grp_1.sort_values(['unique_cust_grp', 'repm_vin', 'repm_work_type', 'repm_ro_dtime'], ascending=[True, 
                                                                                                                  True, True,
                                                                                                                  False])
serv_grp_1['year'] = serv_grp_1['repm_ro_dtime'].dt.year

serv_grp_1['day_diff'] = serv_grp_1.sort_values(['repm_vin', 'repm_ro_dtime'], ascending=[True,False]).groupby(
    ['repm_vin'])['repm_ro_dtime'].apply(lambda x: x - x.shift(-1))
serv_grp_1['day_diff'] = serv_grp_1['day_diff'].dt.days

In [ ]:
serv_grp_1['Repeat_visit_month'] = np.where((serv_grp_1['day_diff']<30), True, False)
serv_grp_1['Repeat_visit_three_month'] = np.where((serv_grp_1['day_diff']<100), True, False)
display(serv_grp_1['Repeat_visit_month'].value_counts())
display(serv_grp_1['Repeat_visit_three_month'].value_counts())

serv_rv_three_flags = serv_grp_1[serv_grp_1['Repeat_visit_three_month']==True].groupby('unique_cust_grp')['repm_ro_dtime'].count().reset_index()
serv_count_one_yr = serv_grp_1[serv_grp_1['repm_ro_dtime']>='2018-06-01'].groupby('unique_cust_grp')['repm_ro_dtime'].count().reset_index()
serv_count_three_yr = serv_grp_1[serv_grp_1['repm_ro_dtime']>='2016-06-01'].groupby('unique_cust_grp')['repm_ro_dtime'].count().reset_index()

serv_rv_yrly_count = serv_grp_1[serv_grp_1['Repeat_visit_month']==True].groupby(['unique_cust_grp', 'year'])['repm_ro_dtime'].count().reset_index()
serv_rv_one_yr_count = serv_rv_yrly_count.groupby('unique_cust_grp').agg({'repm_ro_dtime': ['mean', 'max', 'median', 'min']}).reset_index()
serv_day_diff_count = serv_grp_1.groupby('unique_cust_grp').agg({'day_diff': ['mean', 'max', 'median', 'min']}).reset_index()

def flat(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df
serv_rv_one_yr_count = flat(serv_rv_one_yr_count)
serv_day_diff_count = flat(serv_day_diff_count)

#serv_rv_one_flags_dict = pd.Series(serv_rv_one_flags['repm_ro_dtime'].values, index=serv_rv_one_flags['unique_cust_grp']).to_dict()
serv_rv_three_flags_dict = pd.Series(serv_rv_three_flags['repm_ro_dtime'].values, index=serv_rv_three_flags['unique_cust_grp']).to_dict()
serv_count_one_yr_dict = pd.Series(serv_count_one_yr['repm_ro_dtime'].values, index=serv_count_one_yr['unique_cust_grp']).to_dict()
serv_count_three_yr_dict = pd.Series(serv_count_three_yr['repm_ro_dtime'].values, index=serv_count_three_yr['unique_cust_grp']).to_dict()
#serv_grp_1['rv_3_count'] = serv_grp_1.unique_cust_grp.map(serv_rv_one_flags_dict).fillna(0)
serv_grp_1['rv_3_count'] = serv_grp_1.unique_cust_grp.map(serv_rv_three_flags_dict).fillna(0)
serv_grp_1['serv_count_last_one'] = serv_grp_1.unique_cust_grp.map(serv_count_one_yr_dict).fillna(0)
serv_grp_1['serv_count_last_three'] = serv_grp_1.unique_cust_grp.map(serv_count_three_yr_dict).fillna(0)

serv_rv_feat_1 = serv_grp_1.drop_duplicates(subset=['unique_cust_grp'])[['unique_cust_grp', 'rv_3_count', 'serv_count_last_one',
                                                                        'serv_count_last_three']]
serv_rv_feat_2 = pd.merge(serv_rv_feat_1, serv_rv_one_yr_count, how='left', on=['unique_cust_grp']).rename(columns={
    'repm_ro_dtime mean': 'yearly_rv_3_count mean', 'repm_ro_dtime max': 'yearly_rv_3_count max',
    'repm_ro_dtime min': 'yearly_rv_3_count min', 'repm_ro_dtime median': 'yearly_rv_3_count median'
}).fillna(0)
serv_rv_feat = pd.merge(serv_rv_feat_2, serv_day_diff_count, how='left', on=['unique_cust_grp'])


serv_rv_feat.to_csv('D:/Referral/Cust_grp_splits/repeat_vist/rv_feat_noused_1.dsv', index=False, encoding='utf-8')


**Service dealer history**

In [ ]:
cust_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/cust_grp_noused_1.csv', low_memory=False)
serv_grp_1 = pd.read_csv('D:/Referral/Cust_grp_splits/serv_noused_1.csv', low_memory=False)
#used_cust_grp_1 = used_cust_grp_1

cust_grp_1_dict = pd.Series(cust_grp_1['unique_cust_grp'].values, index=cust_grp_1['repm_vin']).to_dict()
serv_grp_1['unique_cust_grp'] = serv_grp_1.repm_vin.map(cust_grp_1_dict)


serv_grp_1 = serv_grp_1.drop_duplicates(subset=['repm_dlr_code', 'repm_cust_no', 'repm_vin', 'repm_ro_dtime',
                                  'repm_work_type', 'repm_milg'])
display(serv_grp_1.shape)
serv_grp_1['repm_ro_dtime'] = pd.to_datetime(serv_grp_1['repm_ro_dtime'])
serv_grp_1 = serv_grp_1.sort_values(['unique_cust_grp', 'repm_vin', 'repm_work_type', 'repm_ro_dtime'], ascending=[True, 
                                                                                                                  True, True,
                                                                                                                      False])
                                     

def dlr_count(df):
    df['FS_dlr_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='FS')|
                                            (df['repm_work_type']=='UF')].groupby('repm_vin')['repm_dlr_code'].nunique())
    
    df['MS_dlr_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='PS')|
                                            (df['repm_work_type']=='RR')|
                                            (df['repm_work_type']=='RM')].groupby('repm_vin')['repm_dlr_code'].nunique())
    df['AR_dlr_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='AR')|
                                            (df['repm_work_type']=='RA')].groupby('repm_vin')['repm_dlr_code'].nunique())
    df['Total_dlr_count_vin'] = df.repm_vin.map(df.groupby('repm_vin')['repm_dlr_code'].nunique())
    df['FS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='FS')|
                                            (df['repm_work_type']=='UF')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['MS_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='PS')|
                            (df['repm_work_type']=='RR')|
                           (df['repm_work_type']=='RM')].groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['AR_count_vin'] = df.repm_vin.map(df[(df['repm_work_type']=='AR')|
                            (df['repm_work_type']=='RA')].groupby('repm_vin')['repm_ro_dtime'].nunique())    
    df['Total_serv_count_vin'] = df.repm_vin.map(df.groupby('repm_vin')['repm_ro_dtime'].nunique())
    df['FS_dealer_loyalty_vin'] = df['FS_count_vin']/df['FS_dlr_count_vin']
    df['MS_dealer_loyalty_vin'] = df['MS_count_vin']/df['MS_dlr_count_vin']
    df['AR_dealer_loyalty_vin'] = df['AR_count_vin']/df['AR_dlr_count_vin']
    df['Overall_dealer_loyalty_vin'] = df['Total_serv_count_vin']/df['Total_dlr_count_vin']
    
    return df

max_service_dlr = serv_grp_1.groupby('unique_cust_grp')['repm_dlr_code'].apply(max).reset_index().rename(
columns={'repm_dlr_code': 'Max_service_dlr_code'})
max_dlr_dict = pd.Series(max_service_dlr['Max_service_dlr_code'].values, index=max_service_dlr['unique_cust_grp']).to_dict()
serv_grp_1 = dlr_count(serv_grp_1)

serv_grp_1_dlr = serv_grp_1.groupby('unique_cust_grp').agg({'FS_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'MS_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'AR_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                             'repm_dlr_code': pd.Series.nunique,
                                                              'Total_dlr_count_vin': ['mean', 'median', 'min', 'max'],
                                                              'FS_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                               'MS_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                                'AR_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                                'Overall_dealer_loyalty_vin': ['mean', 'median', 'min', 'max'],
                                                             'repm_vin': pd.Series.nunique,
                                                              'repm_ro_dtime': pd.Series.nunique,}).reset_index().rename(
columns={'repm_dlr_code': 'Total_dealer_count', 'repm_ro_dtime': 'Total_service_count'})
def flat(df):
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df
serv_grp_1_dlr = flat(serv_grp_1_dlr)
serv_grp_1_dlr['Max_service_dlr_code'] = serv_grp_1_dlr.unique_cust_grp.map(max_dlr_dict)
serv_grp_1_dlr.to_csv('D:/Referral/Cust_grp_splits/dlr_feat/serv_dlr_feat_1.dsv', encoding='utf-8', index=False)

**Service marketing campaigns feature extraction**

In [ ]:
target_cust_grp = pd.read_csv('D:\Referral\Cust_grp_master_target.dsv', low_memory=False)
camp_data = pd.read_csv('D:\Referral\serv_marketing_data.csv', low_memory=False) #Camp data with camp counts across uniques cut grp
target_cust_grp.unique_cust_grp = target_cust_grp.unique_cust_grp.apply(str)
camp_data['unique_cust_grp'] = camp_data['unique_cust_grp'].apply(str)
test_0 = target_cust_grp[target_cust_grp.Target_final==0].unique_cust_grp.unique().tolist() #Create dict to map target values to camp data
camp_data['Target_binary'] = np.where(camp_data['unique_cust_grp'].isin(test_0), 0, 1) #map target values to camp data

In [ ]:
#Data pre-processing
camp_data_dedup = camp_data.drop_duplicates(subset=['created', 'con_per_id', 'camp_con_num', 'camp_ld_wave_id',
                                                    'msg_bncd_reason_cd', 'msg_bncd_stat_cd', 'stat_cd'])
camp_data_dedup['stat_cd'] = camp_data_dedup['stat_cd'].fillna('NA')
camp_data_dedup = camp_data_dedup[['row_id', 'unique_cust_grp', 'repm_vin', 'stat_cd', 'created']]

In [ ]:
#Feature extraction
camp_data_feat = camp_data_dedup.groupby(['unique_cust_grp']).agg({'row_id': pd.Series.nunique,
                                               'repm_vin': pd.Series.nunique}).reset_index() # Total camp count

feat_1 = camp_data_dedup[camp_data_dedup.stat_cd=='Bounced'].groupby(['unique_cust_grp']).agg({'row_id': pd.Series.nunique}).reset_index().rename(
    columns={'row_id': 'Bounced_count'}) #Bounced email count
feat_2 = camp_data_dedup[camp_data_dedup.stat_cd=='NA'].groupby(['unique_cust_grp']).agg({'row_id': pd.Series.nunique}).reset_index().rename(
    columns={'row_id': 'NA_count'}) #Email count with no status
feat_3 = camp_data_dedup[camp_data_dedup.stat_cd=='Sent'].groupby(['unique_cust_grp']).agg({'row_id': pd.Series.nunique}).reset_index().rename(
    columns={'row_id': 'Sent'}) #Sent email count

# Merge sent, bounced and NA camp count to the main data
camp_data_feat = pd.merge(camp_data_feat, feat_1, on=['unique_cust_grp'], how='left').merge(
    feat_2, on=['unique_cust_grp'], how='left').merge(feat_3, on=['unique_cust_grp'], how='left')

# Merge with target values
camp_data_feat = camp_data_feat.fillna(0)
camp_data_feat['Target_binary'] = np.where(camp_data_feat['unique_cust_grp'].isin(test_0), 0, 1)
camp_data_feat.to_csv('D:/Referral/EDA/Features/camp_data.dsv', encoding='utf-8', index=False)